# Materials API - Exercise 2: Using the MPRester and Pymatgen to Find Materials With Exotic Mechanical Properties

The tetragonal SiO$_2$ polymorph $\alpha$-cristobalite is one of the very few crystalline materials known to have a negative average Poisson's ratio, which means that its cross-section expands under tensile strain rather than contracting. This property can be extremely useful in a variety of applications such as scratch-resistant coatings and high-toughness ceramics. 

Why does $\alpha$-cristobalite exhibit this property while other materials do not? The prevailing hypothesis is that $\alpha$-cristobalite's negative Poisson's ratio is a result of its crystal structure. If that's the case, then perhaps we can find other materials with this exotic property by looking for materials with similar structures and then calculating their Poisson's ratios.

## Step 1: Retrieve the structure of $\alpha$-cristobalite

First, we should find the ground state structure for $\alpha$-cristobalite, which has the spacegroup $P4_12_12$ [92]. Remember, stable materials have *low* energy above hull (`e_above_hull`).

_Hint: Query for the formula and spacegroup for the desired compound and then select the one with the lowest e_above_hull if there are multiple results._

In [1]:
# Your code here
from pymatgen import MPRester

with MPRester("9LO7vrUFbVTMaxdK") as mpr:
    ac = mpr.query({"spacegroup.number": 92, 
                              "pretty_formula": "SiO2", 
                              "e_above_hull": {"$lt": 0.1}}, 
                             ["material_id", "structure"])[0]

In [2]:
ac_structure = ac["structure"]

## Step 2: Build a structure comparison engine and test it out

The code below creates a structure matcher object that can be used to compare if two structures are simlar (loose tolerances = similar, tight tolerances = identical). Please verify that the structure matcher works for identical structure by comparing the structure of AC with itself. 

_Hint: Replace <YOUR CODE HERE> with the two structures you want to compare, separated by a comma._

In [3]:
##### Don't edit code below #####

from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.analysis.structure_matcher import FrameworkComparator
# This line initializes our comparator. Please don't play with its tolerances until after you are done
comparison_engine = StructureMatcher(ltol=.2, stol=.5, angle_tol=10, primitive_cell=True, 
                                     scale=True, attempt_supercell=True, 
                                     comparator=FrameworkComparator())
##### Don't edit code above #####

In [4]:
print(comparison_engine.fit(ac_structure, ac_structure))

True


We know that the high-temperature phase of cristobalite, $\beta$-cristobalite [(mp-546794)](https://materialsproject.org/materials/mp-546794/), has a very similar structure to $\alpha$-cristobalite. Let's see if the structure matcher agrees. Please retreive the structure for `mp-546794` and then compare it with our prototype structure. 

_Hint: No need to re-initialize the structure matcher. Just call it's `fit` function again._

In [5]:
# Your code here
with MPRester("9LO7vrUFbVTMaxdK") as mpr:
    bc_structure = mpr.get_structure_by_material_id("mp-546794")
print(comparison_engine.fit(bc_structure, ac_structure))

True


Just to make sure we haven't increased the tolerances too much, please try it against a random compound to make sure it's not matching dissimilar structures.

_Hint: "mp-4991" is a good random MPID you can use_

In [6]:
# Your code here
# Your code here
with MPRester("9LO7vrUFbVTMaxdK") as mpr:
    bc_structure = mpr.get_structure_by_material_id("mp-4991")
print(comparison_engine.fit(bc_structure, ac_structure))

False


## Step 3: Get a set of candidate structures to compare

Now that we have our comparator, we need some candidates to screen! 

Imagine that we have an experimental colleague, Soren Tsarpinski, who is an expert at synthesizing vanadate compounds. We have a hunch that some of the vanadates coming out of Dr. Tsarpinski's lab might have similar structures to $\alpha$-cristobalite and therefore might have negative Poisson's ratios. Let's see if we're right:

For our search, we want to start with a set of structures that are:
* Computationally tractable, so not too many sites (i.e `nsites` <50 is a good number)
* Not too unlikely to be synthesizable (energy above hull <100 meV, i.e. `{e_above_hull: {"$lt": 0.1}`)
* Have a "vanadate" composition, i.e. `"*V3O8"`

Construct and execute a query to get the structure, MPID, and pretty_formula for all materials that match these criteria:

_Aside; Soren Tsarpinski is an anagram. There is a prize for the first workshop atendee that can guess the original phrase it comes from. (Submit your anwswers on Slack, please.)_

In [7]:
# Your code here
with MPRester("9LO7vrUFbVTMaxdK") as mpr:
    vanadate_ids = [entry["material_id"] for entry in mpr.query('*V3O8', ["material_id"])]
    vanadates = mpr.query({"material_id": {"$in": vanadate_ids}, 
                          "e_above_hull": {"$lt": 0.1}, 
                          "nsites":{"$lt": 50}}, ["material_id", "structure"])

/Users/jdagdelen/opt/anaconda3/envs/mp/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn("No electronegativity for %s. Setting to NaN. "
/Users/jdagdelen/opt/anaconda3/envs/mp/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn("No electronegativity for %s. Setting to NaN. "
/Users/jdagdelen/opt/anaconda3/envs/mp/lib/python3.8/site-packages/pymatgen/core/periodic_table.py:438: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn("No electronegativity for %s. Setting to NaN. "
/Users/jdagde

## Step 4: Screen the vanadates for similar structures to $\alpha$-cristobalite and then check if our hypothesis is valid by querying for their Poisson's ratios. 

Now that we have a list of vanadates, let's screen it for similar structures. After we have the similar structures, make one final query to the Materials API to retrieve the `pretty_formula` and `elasticity.homogenious_poisson` property for each one. 

_Hint: Create an empty list for matches and then iterate through the vanadate entries with a for loop. If its structure is similar to $\alpha$-cristobalite, append its `material_id` to a list of matches. After you have a match list, query the Materials API for entries with a `material_id` that is in your matches list using the `$in` mongodb operator (e.g `{"material_id": {"$in$: matches}}`)_

In [9]:
# Your code here
matches = []
for v in vanadates:
    if comparison_engine.fit(ac_structure, v["structure"]):
        matches.append(v["material_id"])

with MPRester("9LO7vrUFbVTMaxdK") as mpr:
    elastic_data = mpr.query({"material_id": {"$in": matches}}, ['material_id', 'pretty_formula', 'elasticity.homogeneous_poisson'])

for e in elastic_data:
    print(e)

{'material_id': 'mp-775001', 'pretty_formula': 'V3FeO8', 'elasticity.homogeneous_poisson': -0.06}
